In [1]:
# Jake Mileham
# U21692492
# COS 781 A2

In [10]:
# =============================
# imports
#==============================

import pandas as pd
import torch as tch
import torch.nn as nn
import torch.optim as optim
from sklearn.impute import SimpleImputer

In [21]:
# =============================
# Data preprocessing
# =============================
df = pd.read_csv("Almond.csv")
df_copy = df.copy()

# Label encode Almond class

# MAMRA   = 0
# SANORA  = 1
# REGULAR = 2

df_copy["Type"] = df_copy["Type"].map(lambda type : 0 if type == "MAMRA" else 1 if type == "SANORA" else 2)

# Extract class data
df_class = df_copy["Type"]

# Remove features with high amount of missing values
# print(df_copy.isnull().sum())
df_copy.drop(columns=["Unnamed: 0", "Type", "Aspect Ratio", "Eccentricity"], inplace=True)

# df_copy.head()

# Replace missing values by almond class
#TODO: Look into computing mean by class of Almond

imputer = SimpleImputer(missing_values=pd.NA, strategy="mean")
data = imputer.fit_transform(df_copy)


In [38]:
# =============================
# Neural Net
# =============================

# 10 features excluding Aspect Ratio and Eccentricity

features = tch.tensor(data, dtype=tch.float32)
classes = tch.tensor(df_class, dtype=tch.float32).reshape(-1, 1)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.layers = nn.Sequential(
            # Hidden Layer expectes 10 input features
            nn.Linear(10,10),
            nn.ReLU(),
            # Output layer
            nn.Linear(10,1),
            nn.Softmax()  
        )
        

    def forward(self, data):
        return self.layers(data)


network = NeuralNetwork()
loss_func = nn.CrossEntropyLoss()

# We need to test optimizers, but use this as a start
optimiser = optim.Adam(network.parameters(), lr=0.001)

epochs = 100

for epoch in range(epochs):
    prediction = network(features)
    loss = loss_func(prediction, classes)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    print(f'Epoch: {epoch}, Current Loss: {loss}')

prediction = network(features)
accuracy = (prediction.round() == classes).float().mean()
print(f"Accuracy: {accuracy}")